# PROYECTO DE INTELIGENCIA ARTIFICIAL: Seleccion de características

Las siguientes funciones se encargan de obtener los atributos con los que trabajaremos, ademas de obtener los atributos ordenados y los objetivos para ambos csv.

Importaciones:

In [1]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

from sklearn.tree import DecisionTreeClassifier

from matplotlib import pyplot
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer

In [2]:
def lectura_titanic():
    fichero = pd.read_csv('titanic.csv')    
    return fichero

def lectura_breastCancer():
    fichero = pd.read_csv('BreastCancer.csv')
    return fichero

In [3]:
titanic = lectura_titanic()
breast_cancer = lectura_breastCancer()

In [4]:
# Eleccion de semilla inicial aleatoria.
np.random.seed(357823)

# Atributos discretos, continuos y objetivo de titanic.csv.
atributos_discretos_titanic = ['Sex', 'Embarked', 'Alone', 'Deck']
atributos_continuos_titanic = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Initial', 
                           'Age_band', 'Family_Size', 'Fare_cat', 'Title', 'Is_Married']
atributos_titanic = titanic.loc[:, atributos_discretos_titanic + atributos_continuos_titanic]
objetivo_titanic = titanic['Survived']

# Atributos discretos, continuos y objetivo de BreastCancer.csv.
atributos_continuos_breast_cancer = ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 
                           'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 
                           'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error',
                           'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 
                           'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points',
                           'worst symmetry', 'worst fractal dimension']
																				
atributos_breast_cancer = breast_cancer.loc[:, atributos_continuos_breast_cancer]
objetivo_breast_cancer = breast_cancer['diagnosis']

##### A continuación realizaremos los algoritmos de busqueda con los que vamos a trabajar. Para ello, trabajaremos con el algoritmo de "Decision tree clasifier" y el algoritmo de "Naives bayes".

#### Decision tree clasifier

#### Funcion de rendimiento

In [5]:
codificador_atributos_discretos = OrdinalEncoder()
# codificador_atributos_discretos.fit(atributos[atributos_discretos])

atributos = atributos_titanic
atributos_discretos = atributos_discretos_titanic
atributos[atributos_discretos] = codificador_atributos_discretos.fit_transform(atributos[atributos_discretos])

# codificador_objetivo = LabelEncoder()
# objetivo = codificador_objetivo.fit_transform(objetivo)

In [6]:
def rendimiento(atributos, objetivo, feature):
    
    # tree = decision_tree(atributos, objetivo, atributos_discretos)
    clasificador_CART = DecisionTreeClassifier()
    score = cross_validate(clasificador_CART, atributos_titanic[feature], objetivo_titanic, cv = 7, scoring = 'balanced_accuracy')
    return np.mean(score['test_score'])

In [7]:
# feature = ['Embarked', 'Pclass', 'Fare', 'Initial']
feature = ['Initial', 'SibSp', 'Deck', 'Fare_cat', 'Title']
rendimiento(atributos, objetivo_titanic, feature)

0.8144646003484158

In [8]:
def busqueda_secuencial_hacia_atras(atributos, objetivo, n_exp=10, cv=7):
    # Inicialización
    solucion_actual = list(atributos.columns)
    n_features = len(solucion_actual)
    resultados = []

     # Guardar el rendimiento de la solución inicial completa
    rendimiento_inicial = rendimiento(atributos, objetivo, solucion_actual)
    resultados.append((solucion_actual.copy(), len(solucion_actual), rendimiento_inicial))

    
    # Ejecución
    while len(solucion_actual) > 1:
        
        mejor_rendimiento = -float('inf')
        peor_feature = None
        
        # Evaluar todas las posibles soluciones temporales
        for feature in solucion_actual:
            solucion_temporal = solucion_actual.copy()
            solucion_temporal.remove(feature)
            rendimiento_temporal = rendimiento(atributos, objetivo, solucion_temporal)

            print(f"Evaluando sin {feature}: {solucion_temporal} - Rendimiento: {rendimiento_temporal}")

            # Guardar el rendimiento de la peor característica
            if rendimiento_temporal > mejor_rendimiento:
                mejor_rendimiento = rendimiento_temporal
                peor_feature = feature
                
        
        # Actualizar la solución actual eliminando la peor característica
        solucion_actual.remove(peor_feature)

        resultados.append((solucion_actual.copy(), len(solucion_actual), mejor_rendimiento))
    
    # Crear un DataFrame con los resultados
    df_resultados = pd.DataFrame(resultados, columns=['MejorSolucionTemporal', 'Tamaño', 'Rendimiento']).sort_values(by='Rendimiento', ascending=False)
    
    # Devolver los resultados en forma de DataFrame
    return df_resultados

In [153]:
busqueda_secuencial_hacia_atras(atributos, objetivo_titanic)

Evaluando sin Sex: ['Embarked', 'Alone', 'Deck', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Initial', 'Age_band', 'Family_Size', 'Fare_cat', 'Title', 'Is_Married'] - Rendimiento: 0.77336383155293
Evaluando sin Embarked: ['Sex', 'Alone', 'Deck', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Initial', 'Age_band', 'Family_Size', 'Fare_cat', 'Title', 'Is_Married'] - Rendimiento: 0.7651478591158261
Evaluando sin Alone: ['Sex', 'Embarked', 'Deck', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Initial', 'Age_band', 'Family_Size', 'Fare_cat', 'Title', 'Is_Married'] - Rendimiento: 0.7737609684296586
Evaluando sin Deck: ['Sex', 'Embarked', 'Alone', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Initial', 'Age_band', 'Family_Size', 'Fare_cat', 'Title', 'Is_Married'] - Rendimiento: 0.7560786781676087
Evaluando sin Pclass: ['Sex', 'Embarked', 'Alone', 'Deck', 'Age', 'SibSp', 'Parch', 'Fare', 'Initial', 'Age_band', 'Family_Size', 'Fare_cat', 'Title', 'Is_Married'] - Rendimiento: 0.7673255472031629
Evaluan

,MejorSolucionTemporal,Tamaño,Rendimiento
12,"[Sex, Initial, Family_Size]",3,0.803695
13,"[Initial, Family_Size]",2,0.803695
3,"[Sex, Alone, Deck, Pclass, SibSp, Parch, Fare,...",12,0.803090
5,"[Sex, Deck, Pclass, SibSp, Parch, Fare, Initia...",10,0.801842
6,"[Sex, Deck, Pclass, SibSp, Parch, Fare, Initia...",9,0.801090
8,"[Sex, SibSp, Parch, Fare, Initial, Family_Size...",7,0.800418
9,"[Sex, SibSp, Parch, Fare, Initial, Family_Size]",6,0.800418
2,"[Sex, Embarked, Alone, Deck, Pclass, SibSp, Pa...",13,0.799983
4,"[Sex, Alone, Deck, Pclass, SibSp, Parch, Fare,...",11,0.799632
7,"[Sex, Pclass, SibSp, Parch, Fare, Initial, Fam...",8,0.799310


In [ ]:
def busqueda_secuencial_hacia_atras_mixta(atributos, objetivo, atributos_discretos, iteraciones=10, n_exp=10, cv=7):
    # Inicialización
    solucion_actual = list(atributos.columns)
    n_features = len(solucion_actual)
    añadidos = []
    eliminados = []
    resultados = []
    contador = 0

     while len(solucion_actual) > 0:
        rendimiento_actual = rendimiento(atributos, objetivo, solucion_actual, atributos_discretos)
        resultados.append((solucion_actual.copy(), len(solucion_actual), rendimiento_actual))
        
        if len(solucion_actual) == 1:
            break
        
        peor_rendimiento = float('inf')
        peor_feature = None
        
        # Evaluar todas las posibles soluciones temporales
        for feature in solucion_actual:
            solucion_temporal = solucion_actual.copy()
            solucion_temporal.remove(feature)
            rendimiento_temporal = rendimiento(atributos, objetivo, solucion_temporal, atributos_discretos)
            
            # Guardar el rendimiento de la peor característica
            if rendimiento_temporal < peor_rendimiento:
                peor_rendimiento = rendimiento_temporal
                peor_feature = feature
        
        # Actualizar eliminados añadiendo la peor caracteristica y la solución actual eliminando la peor característica
        eliminados.append(peor_feature)
        solucion_actual.remove(peor_feature)

        

    






